In [11]:
%matplotlib inline
import torch
import torchvision
from IPython import display
from matplotlib import pyplot as plt
import numpy as np
import random

In [12]:
def use_svg_display():
    # 用矢量图显示
    display.set_matplotlib_formats('svg')

def set_figsize(figsize=(3.5, 2.5)):
    use_svg_display()
    # 设置图的尺寸
    plt.rcParams['figure.figsize'] = figsize

In [15]:
!ls ./data/iris

iris_test.csv     iris_training.csv


In [55]:
import pandas as pd
class IrisDataSet(torch.utils.data.Dataset):
    def __init__(self, csv_path, sep=','):
        with open(csv_path, 'r') as f:
            lines = f.readlines()[1:]
            datas = [line.strip().split(sep) for line in lines]
            datas = np.array(datas, dtype=np.float32)
            self.featurs = torch.tensor(datas_np[:,:4], dtype=torch.float32)
            self.labels = torch.tensor(datas_np[:,-1], dtype=torch.float32)
    
    def __len__(self):
        return len(self.featurs)
    
    def __getitem__(self, index):
        return self.featurs[index], self.labels[index]
    

In [61]:
train_csv_path = './data/iris/iris_training.csv'
train_data_set = IrisDataSet(train_csv_path)
train_data_iter = torch.utils.data.DataLoader(train_data_set, batch_size=32, shuffle=True)

test_csv_path = './data/iris/iris_test.csv'
test_data_set = IrisDataSet(test_csv_path)
test_data_iter = torch.utils.data.DataLoader(test_data_set, batch_size=32)

print(len(train_data_set), len(test_data_set))

120 120


In [79]:
w = torch.randn((4,3), dtype=torch.float32)
b = torch.zeros((4,1), dtype=torch.float32)

w.requires_grad_(True)
b.requires_grad_(True)

w, b

(tensor([[-1.1504,  1.4084,  0.3405],
         [-0.4739,  0.7614, -0.9388],
         [-1.8759, -0.6740, -0.0252],
         [ 1.0006, -0.8527,  0.3821]], requires_grad=True), tensor([[0.],
         [0.],
         [0.],
         [0.]], requires_grad=True))

In [100]:
def softmax(x):
    return x.exp() / x.exp().sum(dim=1, keepdim=True)

In [80]:
def linear_reg(x):
    return torch.mm(x, w) + b

def cross_entropy(y, y_hat):
    return 

def sgd(params, lr, bs):
    for param in params:
        param.data -= param.grad / bs * lr
        
def net(x):
    return softmax(linear_reg(x))
    

In [97]:
x = torch.tensor([[1.,2.], [2.,5.]])
x

tensor([[1., 2.],
        [2., 5.]])

In [98]:
x.exp(), x.exp().sum(dim=1, keepdim=True)

(tensor([[  2.7183,   7.3891],
         [  7.3891, 148.4132]]), tensor([[ 10.1073],
         [155.8022]]))

In [99]:
x.exp() / x.exp().sum(dim=1, keepdim=True)

tensor([[0.2689, 0.7311],
        [0.0474, 0.9526]])

In [95]:
x.exp()

tensor([[2.7183e+00, 7.3891e+00],
        [7.3891e+00, 2.2026e+04]])

In [96]:
x.exp().sum(dim=1, keepdim=True)

tensor([[1.0107e+01],
        [2.2034e+04]])